In [1]:
!pip install portalocker
!pip install torchmetrics

In [2]:
import argparse
import logging
import time

import torch
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset
from torchtext.data.utils import get_tokenizer, ngrams_iterator
from torchtext.datasets import DATASETS
from torchtext.prototype.transforms import load_sp_model, PRETRAINED_SP_MODEL, SentencePieceTokenizer
from torchtext.utils import download_from_url
from torchtext.vocab import build_vocab_from_iterator
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
import torch.nn.functional as F
from torchtext.vocab import GloVe
from torch.optim import SGD
from torch.optim.lr_scheduler import StepLR
from tqdm import tqdm

torch.autograd.set_detect_anomaly(True)

FILL = '_FILL_'

### Information
- torchtext repo: https://github.com/pytorch/text/tree/main/torchtext
- torchtext documentation: https://pytorch.org/text/stable/index.html

### Constants

In [3]:
DATASET = "AG_NEWS"
DATA_DIR = ".data"
DEVICE = "cpu"
EMBED_DIM = 300
LR = 1.0
BATCH_SIZE = 128
NUM_EPOCHS = 5
PADDING_VALUE = 0
PADDING_IDX = PADDING_VALUE

### Get the tokenizer
- Use the WordLevel tokenizer.


In [4]:
# Get basic tokenizer
basic_english_tokenizer = get_tokenizer("basic_english")

In [5]:
basic_english_tokenizer("This is some text ...")

['this', 'is', 'some', 'text', '.', '.', '.']

In [6]:
# Needed later; set this to the tokenizer above
TOKENIZER = basic_english_tokenizer

### Get the data and get the vocabulary

In [7]:
def yield_tokens(data_iter):
    for _, text in data_iter:
        yield TOKENIZER(text)

In [8]:
train_iter = DATASETS[DATASET](root=DATA_DIR, split="train")
VOCAB = build_vocab_from_iterator(yield_tokens(train_iter), specials=('<pad>', '<unk>'))

# Make the default index the same as that of the unk_token.
VOCAB.set_default_index(VOCAB['<unk>'])

### Get GLOVE embeddings

In [9]:
# Get the GloVe embeddings; this will be slow
GLOVE = GloVe(name='6B', dim=EMBED_DIM)

In [10]:
len(GLOVE), GLOVE.vectors.shape

(400000, torch.Size([400000, 300]))

### Helper functions

In [11]:
def text_pipeline(text):
    return VOCAB(TOKENIZER(text))

def label_pipeline(label):
    return int(label) - 1

Nice link on collate_fn and DataLoader in PyTorch: https://python.plainenglish.io/understanding-collate-fn-in-pytorch-f9d1742647d3

In [20]:
# As before, loop through the batch and transform into tensors
def collate_batch(batch):
    label_list, text_list = [], []
    for (_label, _text) in batch:
        # Get the label from {1, 2, 3, 4} to {0, 1, 2, 3}
        labels = label_pipeline(_label)
        label_list.append(labels)

        # Return a list of ints
        texts = text_pipeline(_text)
        # Append to text_list
        text_list.append(torch.tensor(texts, dtype=torch.int64))

    # Pad and make into tensors as needed
    label_list = torch.tensor(label_list, dtype=torch.int64)
    text_list =  pad_sequence(text_list, batch_first=True, padding_value=PADDING_VALUE)

    return label_list.to(DEVICE), text_list.to(DEVICE)

### Get the data

In [13]:
train_iter = DATASETS[DATASET](root=DATA_DIR, split="train")
num_class = len(set([label for (label, _) in train_iter]))
# What are the classes?
print(f"The number of classes is {num_class} ...")

The number of classes is 4 ...


### Set up the model

Good reference on this type of model
- CNN for Sentence Classification: https://arxiv.org/pdf/1408.5882.pdf

You can build CNN models with either CNN1d or CNN2d.

In [14]:
class CNN1dTextClassificationModel(nn.Module):
    def __init__(
        self,
        vocab_size,
        num_class,
        embed_dim = 300,
        use_pretrained = True,
        fine_tune_embeddings = True,
        debug = False
    ):

        super(CNN1dTextClassificationModel, self).__init__()

        self.embedding = nn.Embedding(
            vocab_size,
            embed_dim,
            padding_idx=PADDING_IDX
        )

        if use_pretrained:
            # Set the embeddings to not requiring gradients since we'll try and modify
            self.embedding.weight.requires_grad = fine_tune_embeddings
            for i in range(vocab_size):
                # Get the token for the index i
                token = VOCAB.get_itos()[i]
                # Modify the embedding for index i by the embedding for that token
                # Do this only if token is in the stoi dictionary for GLOVE
                if token in GLOVE.stoi:
                    self.embedding.weight.data[i] = GLOVE.vectors[GLOVE.stoi[token]]
            self.embedding.weight.requires_grad = False
        else:
            # Otherwise, initialize the weights as specified below
            self.init_weights()

        # If weights do not get changed, turn off gradients for the GloVe embeddings
        if not fine_tune_embeddings:
            self.embedding.weight.requires_grad = False

        # Define a Conv1d layer that collapses all the channels and does not collapse the time dimension
        self.cnn1 = nn.Conv1d(in_channels=embed_dim, out_channels=1, kernel_size=1)

        # Define 3 Conv1d layers each having 1 filter and kernel sizes 2, 3 and 4
        self.cnn2 = nn.Conv1d(in_channels=embed_dim, out_channels=1, kernel_size=2)
        self.cnn3 = nn.Conv1d(in_channels=embed_dim, out_channels=1, kernel_size=3)
        self.cnn4 = nn.Conv1d(in_channels=embed_dim, out_channels=1, kernel_size=4)

        # A linear map from some dimensions to num_class (you need to figure it out)
        self.fc = nn.Linear(in_features=4, out_features=num_class)

        self.debug = debug

    def init_weights(self):
        initrange = 0.5
        # Initialize the embedding weight matrix to uniform between the [-0.5, 0.5]
        self.embedding.weight.data.uniform_(-initrange, initrange)
        # Initialize the weight matrix of fc to uniform between the [-0.5, 0.5]
        self.fc.weight.data.uniform_(-initrange, initrange)
        # Initialize the bias for fc to zero
        self.fc.bias.data.zero_()


    # B = batch_size, L = sequence length, D = vector dimension
    def forward(self, text):

        # B X L X D
        # Get the embeddings for the text passed in
        embedded = self.embedding(text)

        if self.debug:
            print('embedding', embedded.shape)

        # B X D X L
        # Transpose the embedding above as needed
        embedded = embedded.transpose(1, 2)

        # B X 1 X L
        # Pass through cnn1
        cnn1 = self.cnn1(embedded)
        if self.debug:
            print('cnn1', cnn1.shape)

        # B X 1
        # Take Max pooling over time
        cnn1 = F.max_pool1d(cnn1, cnn1.size(2)).squeeze(2)
        if self.debug:
            print('cnn1 after max pool', cnn1.shape)

        # B X 1 X L - 1
        # Pass through cnn2 and add a RELU
        cnn2 =  F.relu(self.cnn2(embedded))
        if self.debug:
            print('cnn2', cnn2.shape)

        # B X 1 X L - 2
        # Pass through cnn3 and add a RELU
        cnn3 =  F.relu(self.cnn3(embedded))
        if self.debug:
            print('cnn3', cnn3.shape)

        # B X 1 X L - 3
        # Pass through cnn4 and add a RELU
        cnn4 =  F.relu(self.cnn4(embedded))
        if self.debug:
            print('cnn4', cnn4.shape)

        # B X 1 in all cases
        # Apply max pooling to each of cnn2, cnn3 and cnn4
        cnn2 = F.max_pool1d(cnn2, cnn2.size(2)).squeeze(2)
        cnn3 = F.max_pool1d(cnn3, cnn3.size(2)).squeeze(2)
        cnn4 = F.max_pool1d(cnn4, cnn4.size(2)).squeeze(2)

        # B X 1 in all cases
        # Apply max pooling over time
        if self.debug:
            print('cnn2 after max', cnn2.shape)

        # Add to each cnn2, 3, 4 a skip connection to cnn1 and average the results
        cnn2 = cnn2 + cnn1
        cnn3 = cnn3 + cnn1
        cnn4 = cnn4 + cnn1
        if self.debug:
            print('cnn2 after skip connection', cnn2.shape)

        # B X 3
        # Concatenate the above
        cnn_concat = torch.cat((cnn1, cnn2, cnn3, cnn4), dim=1)
        
        if self.debug:
            print('cnn concat', cnn_concat.shape)
            # Set the debug to False after the first forward pass
            self.debug = False

        # Pass through an appropriate Linear layer to get the right dimensions needed
        out = self.fc(cnn_concat)

        return out

### Set up the model

In [15]:
# If this is True, we will initialize the Embedding layer with GLOVE
USE_PRETRANED = True,

# If this is True, we will allow for gradient updates on the nn.Embedding layer
FINE_TUNE_EMBEDDINGS = True

# Set the loss appropriately
criterion = nn.CrossEntropyLoss()

model = CNN1dTextClassificationModel(vocab_size=len(VOCAB), num_class=num_class, embed_dim=EMBED_DIM, use_pretrained=USE_PRETRANED, fine_tune_embeddings=FINE_TUNE_EMBEDDINGS)

# Set the optimizer to SGD
# Add an L2 regularizer of 0.00001
optimizer = SGD(model.parameters(), lr=LR, weight_decay=0.00001)

# Set the scheduler to StepLR with gamma=0.1 and step_size = 1.0
scheduler = StepLR(optimizer, step_size=1, gamma=0.1)

### Set up the data

In [21]:
train_iter, test_iter = DATASETS[DATASET]()
train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)

num_train = int(len(train_dataset) * 0.95)
split_train_, split_valid_ = random_split(train_dataset, [num_train, len(train_dataset) - num_train])

train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, collate_fn=collate_batch)

### Train the model

In [22]:
def train(dataloader, model, optimizer, criterion, epoch):
    model.train()
    total_acc, total_count = 0, 0
    total_loss, total_batches = 0.0, 0
    total_zero_gradients_percentage = []
    log_interval = 10

    for idx, (label, text) in tqdm(enumerate(dataloader)):
        # Set gradients to zero
        optimizer.zero_grad()

        # Get the predictions
        predicted_label = model(text)

        # Get the loss
        loss = criterion(predicted_label, label)

        # Do back propagation and get the gradients
        loss.backward()

        # Get the loss per batch and the number of batches
        with torch.no_grad():
            total_loss += loss.item()
            total_batches += 1

        # Loop through all the parameters
        # Specifically, for this batch, get the percentage of zero gradients across all parameters
        # Append this to the list above which will print out the total every 10 batches
        total_nonzero_gradients = 0.0
        total_param_count = 0.0
        for param in model.parameters():
            if param.grad is not None:
                total_nonzero_gradients += torch.count_nonzero(param.grad).item()
                total_param_count += param.numel()

        # Append to total_zero_gradients_percentage
        grads_zero = (1 - (total_nonzero_gradients / total_param_count)) * 100
        total_zero_gradients_percentage.append(grads_zero)

        # Clip the gradient at ?? Should we use 10.0 or 0.1 with the learning rate we picked and the default notebook setting?
        # Use the above loop to help you figure this out
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.1)

        # Do an optimization step
        optimizer.step()

        # Get the accuracy
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += len(label)

        # Log results
        if idx % log_interval == 0 and idx > 0:
            print(
                "| epoch {:3d} | {:5d}/{:5d} batches "
                "| accuracy {:8.3f} "
                "| loss {:8.3f} "
                "| zero gradients percentage {:8.3f}".format(
                    epoch, idx,
                    len(dataloader),
                    total_acc / total_count,
                    total_loss / total_batches,
                    torch.tensor(total_zero_gradients_percentage).mean().item()
                    )
            )
            # Reset variables as needed
            total_acc, total_count = 0, 0
            total_loss, total_batches = 0.0, 0
            total_zero_gradients = []

In [23]:
def evaluate(dataloader, model, criterion):
    model.eval()
    total_acc, total_count = 0, 0
    total_loss = 0.0

    with torch.no_grad():
        for idx, (label, text) in enumerate(dataloader):
            predited_label = model(text)
            loss = criterion(input=predited_label, target=label)
            total_acc += (predited_label.argmax(1) == label).sum().item()
            total_count +=  len(label)
            total_loss += loss.item()
    return total_acc / total_count, total_loss / total_count

In [24]:
for epoch in range(1, NUM_EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader, model, optimizer, criterion, epoch)
    accu_val, loss_val = evaluate(valid_dataloader, model, criterion)
    scheduler.step()
    print("-" * 59)
    print(
        "| end of epoch {:3d} | time: {:5.2f}s | "
        "valid accuracy {:8.3f} ".format(
            epoch,
            time.time() - epoch_start_time,
            accu_val,
            loss_val
            )
    )
    print("-" * 59)

print("Checking the results of test dataset.")
accu_test, loss_test = evaluate(test_dataloader, model, criterion)
print("test accuracy {:8.3f}".format(accu_test))
print("test loss {:8.3f}".format(loss_test))

12it [00:01,  6.86it/s]

| epoch   1 |    10/  891 batches | accuracy    0.301 | loss    1.368 | zero gradients percentage    0.000


22it [00:03,  7.55it/s]

| epoch   1 |    20/  891 batches | accuracy    0.534 | loss    1.187 | zero gradients percentage    0.000


32it [00:04,  8.19it/s]

...

| epoch   5 |   870/  891 batches | accuracy    0.872 | loss    0.382 | zero gradients percentage    0.000


882it [02:00,  8.02it/s]

| epoch   5 |   880/  891 batches | accuracy    0.864 | loss    0.369 | zero gradients percentage    0.000


891it [02:01,  7.32it/s]


| epoch   5 |   890/  891 batches | accuracy    0.882 | loss    0.363 | zero gradients percentage    0.000
-----------------------------------------------------------
| end of epoch   5 | time: 126.45s | valid accuracy    0.870 
-----------------------------------------------------------
Checking the results of test dataset.
test accuracy    0.833
test loss    0.004
